# Focal Loss 损失函数

> 论文地址：https://arxiv.org/abs/1708.02002
<br>
> 代码地址：https://github.com/facebookresearch/Detectron
<br>
> 参考博文：https://blog.csdn.net/PLANTTHESON/article/details/133933456


## 1.简介

**Focal Loss** 是一个在目标检测领域常用的损失函数，它是何凯明大佬在RetinaNet网络中提出的，解决了目标检测中正负样本极不平衡和难分类样本学习的问题。<br> 
**Focal Loss** 通过引入一个可调节的聚焦参数，将容易分类的样本的权重降低，而将困难样本的权重提高，从而使得模型更加关注困难样本，提高了模型的性能。<br>
需要注意的是，Focal Loss损失函数容易受到**噪声**的干扰，因此训练集中标注的信息尽量**不要出现错误**的情况。

                        


## 2. 相关背景理论知识

### 2.1 交叉熵

在信息学中 **信息熵(entropy)** 是表示系统的混乱程度和确定性的。**交叉熵（Cross Entropy）** 是一种衡量两个概率分布之间的相似程度的指标。在机器学习中，交叉熵常常用于评估模型预测结果与真实标签之间的差距。交叉熵越小，说明模型预测结果与真实标签越接近，模型性能越好。<br>
交叉熵公式如下：
$$
H(p,q) = -\sum_{i=1}^{n}p(x_i)\log q(x_i)
$$

其中，$p$ 是真实分布，$x_i$ 是预测样本，$q$ 是预测分布，$n$ 是类别数。<br>


### 2.1.1 二分类交叉熵

对于二分类问题来说，他的交叉熵是：
\begin{aligned}
    \\\mathrm{CE_{loss}}(p,y)=\begin{cases}-\log(p)&\text{if } y=1
    \\-\log(1-p)&\text{otherwise}.\end{cases}
\end{aligned}
或者是：
$$CE_{loss} = -[y \cdot log(p) + (1-y) \cdot log(1-p)]$$
其中，p表示y=1的概率，接下来我们来定义 p 的函数：<br>
\begin{aligned}p_{\mathrm t}=\begin{cases}p&\text{if }y=1\\1-p&\text{otherwise},\end{cases}\end{aligned}
那么，交叉熵可以表示为：<br>
\begin{aligned}\mathrm{CE_{loss}}(p,y)=\mathrm{CE_{loss}}(p_\mathrm{t})=-\log(p_\mathrm{t})\end{aligned}

### 2.1.2 多分类交叉熵

多分类交叉熵公式如下：<br>
$$

CE_{loss} = - \sum_{i}^{C} [y_i \cdot log(p_i) + (1-y_i) \cdot log(1-p_i)]
$$

<!-- 如果看作只计算预测正确的概率的交叉熵，那么公式可以简化为：<br>
$$
CE_{loss} = - \sum_{i}^{C} log(p_i)
$$ -->
- $C$ 是类别数
- $y_i$ 是第 $i$ 类的标签，如果第 $i$ 类是正样本，那么 $y_i=1$，否则 $y_i=0$
- $p_i$ 是第 $i$ 类的预测正确的概率
$ 0 < p_i < 1$

### 2.2 Balanced Cross Entropy

通过引入平衡因子α来平衡正负样本的权重。**平衡因子α** （α ∈ [ 0 , 1 ] ）的值通常设置为正样本的数量与负样本数量之间的比值。<br>
在计算损失时，对于正样本，其损失权重为1；对于负样本，其损失权重为α。通过这种方式，Balanced Cross Entropy可以平衡正负样本的权重，从而提高模型的性能。

计算公式：<br>
\begin{aligned}\mathrm{CE_{loss}}(p_\mathrm{t})=-\alpha_\mathrm{t}\log(p_\mathrm{t})\end{aligned}
对于 α 设定不同值的效果：<br>
<center>

![img](./imgs/BCE_α.png)
</center>
论文中当 α=0.75 时，效果最好。

## 3. Focal Loss


Focal Loss 是在 Balanced Cross Entropy 的基础上，进一步引入了 **聚焦参数γ**（γ ∈ [ 0 , 5 ]）来调节损失函数的权重。聚焦参数 **γ** 的引入，使得模型更加关注那些被错误分类的样本，从而提高了模型的性能。<br>
计算公式如下：<br>
\begin{equation}
    \mathrm{FL_{loss}}(p_\mathrm{t})=-\alpha_\mathrm{t}(1-p_\mathrm{t})^\gamma\log(p_\mathrm{t}). 
\end{equation}

由于$$CE_{loss}(p_t) = -log(p_t)$$ 可得：$$p_t = e^{-CE_{loss}} $$所以：$$FL_{loss}(p_t) = {\alpha}(1-e^{-CE_{loss}})^{\gamma}(CE_{loss})$$
$(1-p_{\mathrm{t}})^{\gamma} $ 可以减低易分样本的损失贡献，从而增加难分样本的损失比例，解释如下：当Pt趋向于1，即说明该样本是易区分样本，此时调制因子 $(1-p_{\mathrm{t}})^{\gamma} $ 是趋向于0，说明对损失的贡献较小，即减低了易区分样本的损失比例。当 pt 很小，也就是假如某个样本被分到正样本，但是该样本为前景的概率特别小，即被错分到正样本了，此时 调制因子 $(1-p_{\mathrm{t}})^{\gamma} $ 是趋向于1，对loss也没有太大的影响。

- Focal Loss 的效果：

<table style="width: 100%; table-layout: fixed;">
    <tr>
        <td style="text-align: center;"><img width="400px" src="./imgs/FL_γ.png"></td>
        <td style="text-align: center;"><img width="400px" src="./imgs/FL_γ_α.png"></td>
        <td style="text-align: center;"><img width="400px" src="./imgs/COCO_AP_FL.png"></td>
    </tr>
    <tr>
        <td style="text-align: center;"> 不同的 γ 值的效果</td>
        <td style="text-align: center;"> γ 和 α 对比试验</td>
        <td style="text-align: center;"> 不同模型使用FL后，在COCO数据集上的AP表现 </td>
    </tr>
</table>

## 4. Focal Loss 代码实现

In [64]:
import torch
import torch.nn as nn
import torch.nn.functional as F

output_tensor = torch.rand(1, 4, 128, 128, 128)

target_tensor = torch.randint(0, 4, (1, 128, 128, 128))

output_onehot = F.one_hot(target_tensor, num_classes=4).permute(0, 4, 1, 2, 3)

output_tensor.shape, target_tensor.shape, output_onehot.shape
output_tensor.max(), target_tensor.max(), target_tensor.unique(), output_onehot.unique()


(tensor([0.0000e+00, 5.9605e-08, 5.9605e-07,  ..., 1.0000e+00, 1.0000e+00,
         1.0000e+00]),
 tensor([0, 1, 2, 3]),
 tensor([0, 1]))

In [61]:
class FocalLoss(nn.Module):
    def __init__(self, alpha=0.25, gamma=2):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma

    def forward(self, pred, targets):
        # 计算交叉熵损失
        celoss = F.cross_entropy(pred, targets, reduction='mean')

        # 计算调制因子
        pt = torch.exp(-celoss)
        modulating_factor = (1 - pt) ** self.gamma

        # 计算加权损失
        loss = self.alpha * modulating_factor * celoss

        return loss.mean()
    
focal_loss = FocalLoss(alpha=0.25, gamma=2)
loss = focal_loss(output_tensor.float(), target_tensor)
print(loss)

RuntimeError: Expected floating point type for target with class probabilities, got Long